In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import json
import requests

In [3]:
req_list = [
    'Mean travel time to work (minutes)',
    'Median earnings (dollars)',
    
]

In [4]:
def clean(x):
    try:
        x = str(x)
        x = x.replace(',','').replace('%','')
        x = float(x)
        return x
    except:
        return x
def extract_data(file):
    df  = pd.read_csv(file)
    df = df[[i for i in df.columns if  'Total!!Estimate' in i or 'Label' in i]]
    df = df.T
    df.columns = df[:1].values[0]
    df  = df[1:]
    df.columns = [i.replace('\xa0','') for i in df.columns]
    final = df[req_list]
    for cols in final.columns:
        final[cols] = final[cols].map(lambda x: clean(x))
    return final
    
    

In [7]:
files = os.listdir('./transport-data')
files = ['./transport-data/'+ file for file in files if '.csv' in file]

In [8]:
from tqdm import tqdm
data = pd.DataFrame()
for file in tqdm(files):
    d = extract_data(file)
    year = file.split('.')[1][-4:]
    d['year'] = year
    data = pd.concat([data,d],axis=0)
    
    

  0%|                                                                                                                                          | 0/5 [00:00<?, ?it/s]/var/folders/3m/q31sgtj91_n394m9656x0bwh0000gp/T/ipykernel_63897/473586990.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[cols] = final[cols].map(lambda x: clean(x))
/var/folders/3m/q31sgtj91_n394m9656x0bwh0000gp/T/ipykernel_63897/473586990.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[cols] = final[cols].map(lambda x: c

In [9]:
data = data.reset_index()
data = data.rename(columns={'index':'Metro'})

In [10]:

data['Metro']  = data.Metro.map(lambda metro: metro.split(',')[0])


In [11]:
metro_mapper = {
    'Miami-Fort Lauderdale-West Palm Beach':'Miami-Fort Lauderdale-Pompano Beach',
    'Houston-Pasadena-The Woodlands':'Houston-The Woodlands-Sugar Land',
    'Denver-Aurora-Centennial':'Denver-Aurora-Lakewood',
    'Austin-Round Rock-San Marcos':'Austin-Round Rock-Georgetown',
    'Atlanta-Sandy Springs-Roswell':'Atlanta-Sandy Springs-Alpharetta'}


In [12]:
data['Metro'] = data.Metro.map(lambda x: metro_mapper[x] if metro_mapper.get(x) is not None else x)

In [17]:
metros = pd.read_csv('Metros.csv')
data = pd.merge(metros, data)
data

,geoID,Metro,Mean travel time to work (minutes),Median earnings (dollars),year
0,1,Atlanta-Sandy Springs-Alpharetta,30.9,50193.0,2022
1,1,Atlanta-Sandy Springs-Alpharetta,31.6,50722.0,2023
2,1,Atlanta-Sandy Springs-Alpharetta,29.6,45903.0,2021
3,1,Atlanta-Sandy Springs-Alpharetta,32.5,41482.0,2019
4,1,Atlanta-Sandy Springs-Alpharetta,32.1,41916.0,2020
...,...,...,...,...,...
90,19,Tampa-St. Petersburg-Clearwater,28.5,44126.0,2022
91,19,Tampa-St. Petersburg-Clearwater,29.4,46426.0,2023
92,19,Tampa-St. Petersburg-Clearwater,26.7,41516.0,2021
93,19,Tampa-St. Petersburg-Clearwater,28.3,36725.0,2019


In [18]:
data.to_csv('transportData.csv',index=False)

In [15]:
files

['./transport-data/ACSST1Y2022.S0802-2024-11-10T171845.csv',
 './transport-data/ACSST1Y2023.S0802-2024-11-10T171825.csv',
 './transport-data/ACSST1Y2021.S0802-2024-11-10T171859.csv',
 './transport-data/ACSST1Y2019.S0802-2024-11-10T171926.csv',
 './transport-data/ACSST5Y2020.S0802-2024-11-10T171913.csv']